In [1]:
import sqlite3

'''
一共開 3 個 DB
1. 存 NER 結果
2. 存 POS 結果
3. 存 RE 結果: 含網址、Email、Username、(行動、市內)電話、IP
'''

con_NER = sqlite3.connect("JudicialAnalysis_NER.db")
cur_NER = con_NER.cursor()

con_POS = sqlite3.connect("JudicialAnalysis_POS.db")
cur_POS = con_POS.cursor()

con_RE = sqlite3.connect("JudicialAnalysis_RE.db")
cur_RE = con_RE.cursor()

In [2]:
# 創建 Table
new = 0
if new:
    cur_NER.execute("CREATE TABLE results(jid, date, keyword_type, keywords)")
    con_NER.commit()
    con_NER.close()
    
    cur_POS.execute("CREATE TABLE results(jid, date,  keyword_type, keywords)")
    con_POS.commit()
    con_POS.close()
    
    cur_RE.execute("CREATE TABLE results(jid, date, keyword_type, keywords)")
    con_RE.commit()
    con_RE.close()

In [3]:
# 把原本裁判書檔按順序反過來
# 從 2023年12月開始
reverse = 0 # 已經做過就不用再做了
if reverse:
    import os 

    source_file = os.path.join(
            'C:\\', '課程', '交大','論文','案件整理',
            'json_files.txt'
            )

    with open(source_file, 'r', encoding='utf-8') as f:
        # 2022~2023年判決書有211萬7,454個json檔
        # 故一次讀一個處理，減少記憶體占用
        json_files_R = f.read().strip().split('\n')

    json_files_R.sort(reverse=True)
    json_files_R = [f for f in json_files_R if os.path.isfile(f)]

    source_file_R = os.path.join(
            'C:\\', '課程', '交大','論文','案件整理',
            'json_files_R.txt'
            )
    
    json_files_R_STR = '\n'.join(json_files_R).strip()
    
    with open(source_file_R, 'w', encoding='utf-8') as f:
        # 2022~2023年判決書有211萬7,454個json檔
        # 故一次讀一個處理，減少記憶體占用
        f.read().strip().split('\n')

In [4]:
import re
from valid_IP import valid_IPv4s, valid_IPv6s

# 用來抓網址s、Domains
def extract_urls(text):
    urls = []
    patterns = [r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', r'https?://([\w\.\-]+)']
    for url_pattern in patterns:
        urls += re.findall(url_pattern, text)
    urls = list(set(urls))
    urls.sort()
    return urls

# 用來抓 Emails
def extract_emails(text):
    email_pattern = r'[\w.-]+@[\w.-]+'
    emails = re.findall(email_pattern, text)
    emails = list(set(emails))
    emails.sort()
    return emails

# 用來抓 Usernames
# \w : 包括(alphanumeric) [0-9a-zA-Z_]
# 後來發現 \w 在 python3 是 unicode 也支持中文
# 解決方法: 要在 re.findall 第三個參數加入 re.A 恢復使用 ASCII，例如: re.findall(r"\w", text, re.A))
# 我是乾脆直接指定 [0-9a-zA-Z_]
def extract_usernames(text):
    # 用戶名模式：開頭可能有@，接著是字母、數字、下劃線、點或破折號
    username_pattern = r'@?[a-zA-Z][0-9a-zA-Z_.-]+'
    usernames = re.findall(username_pattern, text)
    usernames = list(set(usernames))
    usernames.sort()
    return usernames

# 用來抓手機號碼
def extract_mobilephones(text):
    # 手機可能有 - 也可能沒有 -
    # 國際號碼可能有 886
    cellphones = []
    cellphone_patterns = [r'(\+?886\d{3}-?\d{3}-?\d{3})', r'(0\d{3}-?\d{3}-?\d{3})']
    for cellphone_pattern in cellphone_patterns:
        cellphones += re.findall(cellphone_pattern, text)
    cellphones = list(set(cellphones))
    cellphones.sort()
    return cellphones

# 用來抓市內電話號碼
# 規則參考: https://zh.wikipedia.org/zh-tw/%E4%B8%AD%E8%8F%AF%E6%B0%91%E5%9C%8B%E9%9B%BB%E8%A9%B1%E8%99%9F%E7%A2%BC
def extract_telephones(text):
    telephone_pattern = r'\(?0[2-8]\d{0,3}\)?-?\d{1,4}-?\d{4}'
    telephones = re.findall(telephone_pattern, text)
    telephones = list(set(telephones))
    telephones.sort()
    return telephones

# 用來抓 IP
def extract_ip(text):
    v4a = valid_IPv4s(text)
    v6a = valid_IPv6s(text)
    
    # 結合兩者結果
    ips = list(set(v4a + v6a))
    ips.sort()
    return ips

# 用來初步抓罪名
def extract_crime(text):
    crimes = []
    patterns = [r'違反(\S+(?=[等違反]))等?[案罪]', r'因(\S+(?=[等犯而]))等?[案罪]', r'因(\S+)等?案', r'號(\S+)等?案', r'違反(\S+)等?案件']
    for pattern in patterns:
        crimes += re.findall(pattern, text)
    crimes = [c.replace('違反', '').replace('等', '') for c in crimes if len(c) < 15 and len(c) > 1 ]
    
    crimes = [c for c in crimes if c]
    crimes = [c for c in crimes if not str(c[0]).isdigit()]
    crimes = [c for c in crimes if not '刑事判決' in c and not '聲請' in c and not ',' in c and not '起訴' in c]
    crimes = [c for c in crimes if c]
    
    crimes = list(set(crimes))
    crimes.sort()
    
    return crimes

re_extractors = {
    'crimes' : extract_crime,
    'urls' : extract_urls,
    'emails' : extract_emails,
    'usernames' : extract_usernames,
    'mobilephones' : extract_mobilephones,
    'telephones' : extract_telephones,
    'ips' : extract_ip,
}

In [5]:
from ElementExtractor import Judicial_Parser
import os, time, traceback, json, re

In [6]:
from ckiptagger import data_utils, construct_dictionary, WS, POS, NER

In [7]:
# Downloads to ./data.zip (2GB) and extracts to ./data/
# data_utils.download_data_url("./") # iis-ckip
#data_utils.download_data_gdown("./") # gdrive-ckip 已下載就不用再下載

In [8]:
# To use GPU:
#    1. Install tensorflow-gpu (see Installation)
#    2. Set CUDA_VISIBLE_DEVICES environment variable, e.g. os.environ["CUDA_VISIBLE_DEVICES"] = "0"
#    3. Set disable_cuda=False, e.g. ws = WS("./data", disable_cuda=False)

# 採坑心得
# 新版已經沒有 tensorflow-gpu
# 我使用 python 3.7 的環境 (conda create -n ckipcore python=3.7)
# 先安裝 pip install -U ckiptagger[tf,gdown]
# 再改用 Conda install tensorflow 去安裝匹配的新版
# 搭配 disable_cuda=False 參數，從 windows 工作管理員之效能頁籤看起來，確實有使用 GPU

ws = WS("./data", disable_cuda=False)
pos = POS("./data", disable_cuda=False)
ner = NER("./data", disable_cuda=False)

C:\ProgramData\Anaconda3\envs\ckipcore\lib\site-packages\ckiptagger\model_ws.py:106: UserWarning: `tf.nn.rnn_cell.LSTMCell` is deprecated and will be removed in a future version. This class is equivalent as `tf.keras.layers.LSTMCell`, and will be replaced by that in Tensorflow 2.0.
  cell = tf.compat.v1.nn.rnn_cell.LSTMCell(hidden_d, name=name)
C:\ProgramData\Anaconda3\envs\ckipcore\lib\site-packages\ckiptagger\model_pos.py:56: UserWarning: `tf.nn.rnn_cell.LSTMCell` is deprecated and will be removed in a future version. This class is equivalent as `tf.keras.layers.LSTMCell`, and will be replaced by that in Tensorflow 2.0.
  cell = tf.compat.v1.nn.rnn_cell.LSTMCell(hidden_d, name=name)
C:\ProgramData\Anaconda3\envs\ckipcore\lib\site-packages\ckiptagger\model_ner.py:57: UserWarning: `tf.nn.rnn_cell.LSTMCell` is deprecated and will be removed in a future version. This class is equivalent as `tf.keras.layers.LSTMCell`, and will be replaced by that in Tensorflow 2.0.
  cell = tf.compat.v1.n

In [9]:
# 簡繁轉換器 (暫時用不到)
# import opencc
# converter = opencc.OpenCC('t2s.json')

In [10]:
from IPython.display import clear_output

start_time = time.time()
print(f'[現在時間] {time.strftime("%Y-%m-%d %H:%M:%S")}')

source_file = os.path.join(
        'C:\\', '課程', '交大','論文','案件整理',
        'json_files.txt'
        )
    
count_file_done_num = 0
clear_output()

# 多工時使用
skip = 0 # 要跳過從第幾開始

print('-'*50)
print(f'>>開始分析...')
print(f'[當前已分析數量] {count_file_done_num}')

with open(source_file, 'r', encoding='utf-8') as f:
    # 2022~2023年判決書有211萬7,454個json檔
    # 故一次讀一個處理，減少記憶體占用
    raw_data = ''
    
    while 1: 
        # 開始逐案讀取資料分析
        file = f.readline()
        if '\\2023' in file: # 因為數量太多要跑太久，先從2023開始好了
            count_file_done_num += 1

            if '刑事' in file:
                # 篩掉一些比較非刑案的
                if not ('簡易' in file or '民事' in file or '補償' in file or '憲法' in file or '商業' in file):
                    # 開始讀取裁判書內容
                    file = file.strip()

                    # 顯示進度
                    temp_time = time.time()
                    elapsed_time = temp_time - start_time
                    hours = int(elapsed_time // 3600)  # 計算小時數
                    minutes = int((elapsed_time % 3600) // 60)  # 計算分鐘數
                    seconds = int(elapsed_time % 60)  # 計算剩餘秒數

                    clear_output()
                    print('-'*50)
                    print(f'>>開始分析...')
                    print(f'[當前已分析數量] {count_file_done_num}')
                    print(f'[累計耗時] {hours}:{minutes}:{seconds}')
                    print(f'[當前檔案] {file}')

                    # 確定 file 存在
                    if not os.path.isfile(file):
                        print(f'[並非正常檔案] 跳過...')
                        time.sleep(1)
                        continue

                    try:
                        with open(file, 'r', encoding='utf-8') as jf:
                            json_data = json.load(jf)
                    except:
                        x = traceback.format_exc()
                        print(x)
                        time.sleep(3)
                        continue

                    # 資料清洗
                    parser = Judicial_Parser(json_data)

                    jid = parser.jid.strip().replace(' ', '')
                    date = parser.dateSTR.strip().replace(' ', '')
                    raw = parser.raw
                    text = parser.text
                    text_u = parser.text_u

                    # 確定是否已經在資料庫內
                    check_res = cur_POS.execute(f"SELECT jid FROM results WHERE jid='{jid}'")
                    temp = check_res.fetchone()
                    if temp:
                        print(f'[JID] {jid}')
                        print('>>此JID發現資料庫已有，跳過')
                        continue
                        
                    print('-'*50)
                    print(text_u)
                    print('-'*50)

                    # 沒有在資料庫內才處理

                    '''
                    ------------------------------------------------------------------
                    RE 擷取
                    ------------------------------------------------------------------
                    '''
                     # Input text
                    print('[RE] 分析中')
                    # 擷取 RE 結果
                    re_dict = dict()
                    for re_type, re_func in re_extractors.items():
                        re_temp_result = re_func(text_u)
                        if re_temp_result:
                            re_dict[re_type] = re_temp_result
                        print(f'>> [{re_type}] {re_temp_result}')

                    '''
                    ------------------------------------------------------------------
                    POS & NER 擷取
                    ------------------------------------------------------------------
                    '''
                    # Input text
                    print('-'*50)
                    print('[WS] 分析中')
                    sentence_list = text_u.split('。')
                    word_sentence_list = ws(
                        sentence_list,
                        # sentence_segmentation = True, # To consider delimiters
                        # segment_delimiter_set = {",", "。", ":", "?", "!", ";"}), # This is the defualt set of delimiters
                        # recommend_dictionary = dictionary1, # words in this dictionary are encouraged
                        # coerce_dictionary = dictionary2, # words in this dictionary are forced
                    )
                    print('[POS] 分析中')
                    pos_sentence_list = pos(word_sentence_list)
                    print('[NER] 分析中')
                    entity_sentence_list = ner(word_sentence_list, pos_sentence_list)

                    # 整理 POS 結果
                    pos_dict = dict()
                    for idx, words in enumerate(word_sentence_list):
                        for idx2, word in enumerate(words):
                            temp_type = pos_sentence_list[idx][idx2]
                            word = word.strip().replace(' ', '').replace(':', '').replace(',', '')
                            pos_dict.setdefault(temp_type, set())
                            pos_dict[temp_type].add(word)

                    # 擷取 NER 結果
                    ner_dict = dict()
                    for e in entity_sentence_list:
                        for ee in e:
                            ner_type = ee[2]
                            ner_dict.setdefault(ner_type, set())
                            temp_ner_result = ee[3].strip().replace(' ', '').replace(':', '').replace(',', '')
                            ner_dict[ner_type].add(temp_ner_result)
                    print('-'*50)

                    '''
                    ------------------------------------------------------------------
                    寫入 DB Table
                    ------------------------------------------------------------------
                    '''
                    print('-'*50)
                    print('>>準備寫入 RE 結果')
                    for re_type, re_keywords in re_dict.items():
                        values_str = ','.join(re_keywords)
                        to_db_content = "INSERT INTO results VALUES(?, ?, ?, ?)" # ? 是一個參數占位符，代表將要插入的值。
                        cur_RE.execute(to_db_content, (jid, date, re_type, values_str))
                    con_RE.commit()

                    print('-'*50)
                    print('>>準備寫入 POS 結果')
                    for temp_type, temp_keywords in pos_dict.items():
                        temp_keywords = {tk.strip().replace(' ', '').replace(',', '').replace(':', '') for tk in temp_keywords if len(tk.strip().replace(' ', '').replace(',', '').replace(':', '')) > 1}
                        temp_keywords = list(temp_keywords)
                        temp_keywords.sort()
                        values_str = ','.join(temp_keywords)
                        to_db_content = "INSERT INTO results VALUES(?, ?, ?, ?)" # ? 是一個參數占位符，代表將要插入的值。
                        cur_POS.execute(to_db_content, (jid, date, re_type, values_str))
                    con_POS.commit()

                    print('-'*50)
                    print('>>準備寫入 NER 結果')
                    for temp_type, temp_keywords in ner_dict.items():
                        temp_keywords = {tk.strip().replace(' ', '').replace(',', '').replace(':', '') for tk in temp_keywords if len(tk.strip().replace(' ', '').replace(',', '').replace(':', '')) > 1}
                        temp_keywords = list(temp_keywords)
                        temp_keywords.sort()
                        values_str = ','.join(temp_keywords)
                        to_db_content = "INSERT INTO results VALUES(?, ?, ?, ?)" # ? 是一個參數占位符，代表將要插入的值。
                        cur_NER.execute(to_db_content, (jid, date, re_type, values_str))
                    con_NER.commit()

                    print('-'*50)
                    print('>>完成!!!')

                
# 關閉 DB占用
con_RE.close()
con_POS.close()
con_NER.close()

# 顯示統計結果
end_time = time.time()
elapsed_time = end_time - start_time
hours = int(elapsed_time // 3600)  # 計算小時數
minutes = int((elapsed_time % 3600) // 60)  # 計算分鐘數
seconds = int(elapsed_time % 60)  # 計算剩餘秒數
print(f'[累計耗時] {hours}:{minutes}:{seconds}')

--------------------------------------------------
>>開始分析...
[當前已分析數量] 3707
[累計耗時] 0:1:6
[當前檔案] C:\課程\交大\論文\案件整理\202301\最高法院刑事\TPSM,110,台上,5654,20230111,1.json
[RE] 分析中
>> [crimes] []
>> [urls] []
>> [emails] []
>> [usernames] []
>> [mobilephones] []
>> [telephones] []
>> [ips] []
--------------------------------------------------
[WS] 分析中
[POS] 分析中
[NER] 分析中


KeyboardInterrupt: 

In [11]:
text_u

'最高法院刑事判決110年度台上字第5654號110年度台上字第5655號 \n上 訴 人 \n (被 告) 許玉秀 \n選任辯護人 吳澄潔律師上 訴 人 \n (被 告) 劉富榮 \n選任辯護人 楊靖儀律師上 訴 人 \n (被 告) 傅 中 \n選任辯護人 黃燦堂律師上 訴 人 \n (被 告) 劉信宏 \n選任辯護人 陳怡融律師上 訴 人 \n (被 告) 李鎮衛 \n選任辯護人 王勝彥律師 陳魁元律師上 訴 人\n (參與人) 三山國王宮 \n代 表 人 邱善龍 \n上列上訴人等因被告等違反貪污治罪條例案件,不服臺灣高等法院高雄分院中華民國110年5月12日第二審判決(109年度上訴字第416、417號,起訴及追加起訴案號:臺灣屏東地方檢察署105年度偵字第8509、6714、9531號,106年度偵字第6634、8123、8427號),提起上訴,本院判決如下。主文 原判決關於許玉秀、劉富榮、傅中、劉信宏、李鎮衛如其事實欄 二部分,暨參與人三山國王宮部分均撤銷,發回臺灣高等法院高雄分院。其他上訴駁回。理由 壹、撤銷發回(許玉秀、劉富榮、傅中、劉信宏、李鎮衛關於原判決事實欄《下稱事實欄》二及三山國王宮)部分。一、本件原審審理結果,認定上訴人即被告許玉秀、傅中、李鎮衛為公務員,與劉富榮、劉信宏(下或稱許玉秀等5人)有事實欄二相關所載違反貪污治罪條例、商業會計法、偽造文書犯行,因而撤銷第一審該部分科刑及就上訴人即參與人三山國王宮諭知沒收、追徵之判決,改判仍依想像競合犯,從一重論處許玉秀等5人共同犯或與公務員共同犯利用職務機會詐取財物罪刑,並諭知相關沒收、追徵。固非無見。二、惟查。㈠憲法第16條規定保障人民之訴訟權,就刑事被告而言,包括其在訴訟上應享有充分之防禦權(司法院釋字第582號解釋意旨參照)。國家經由刑事審判程序,對被告之特定犯罪事實,決定國家刑罰權存在與否及具體內容,應兼顧實體上發現真實及程序上公平正當。又刑事訴訟法第95條第1項第1款規定,訊問被告應先告知犯罪嫌疑及所犯所有罪名。罪名經告知後,認為應變更者,應再告知。所謂「犯罪嫌疑及所犯所有罪名」,除起訴書所記載之犯罪事實及所犯法條外,自包含依刑事訴訟法第267條規定起訴效力所擴張之犯罪事實及罪名,暨依同法第300條規定變更起訴法條後之新罪名,而同法第288條第3項後段所謂「審判長就被告被訴事實為訊問者」

In [ ]:
等因被告等違反貪污治罪條例案件

In [17]:
#patterns = [r'違反(\S+(?=[等違反]))等?[案罪]', r'因(\S+(?=[等犯而]))等?[案罪]', r'因(\S+)等?案', r'號(\S+)等?案']
re.findall(r'違反(\S+)等?案件', text_u)

['貪污治罪條例']